https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb

https://www.youtube.com/watch?v=LNwODJXcvt4&list=PLT4ZwFPi5zxFftzI735YoLmq3bFiP2DBo&index=40


Todos:
* Change the next roboflow dataset to have warriors-home, cavs-away, ball to basketball, rim to hoop, and leave referee


In [ ]:

#from roboflow import Roboflow
#rf = Roboflow(api_key="htpcxp3XQh7SsgMfjJns")
#project = rf.workspace("amrita-hlhw6").project("basketball-and-hoop-detection")
#dataset = project.version(1).download("yolov8")

from roboflow import Roboflow
rf = Roboflow(api_key="htpcxp3XQh7SsgMfjJns")
project = rf.workspace("basketball-formations").project("warriors-vs-cavs-2016")
dataset = project.version(10).download("yolov8")
#v5 is only basketball, hoop, referee, and people. warriors/cavs need a lot more images annotated


In [ ]:
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

import torch
from torch import nn

# Note: this notebook requires torch >= 1.10.0
torch.__version__

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo mode-checks

In [ ]:
#use to update yaml file train and val to just train/images because the extra hoop-detection is throwing it off

import yaml

def update_yaml_file(path):
    # Load the yaml file
    with open(path, 'r') as file:
        data_yaml = yaml.safe_load(file)

    # Modify the train and validation paths
    data_yaml['train'] = 'train/images'
    data_yaml['val'] = 'valid/images'

    # Save the modified yaml file back
    with open(path, 'w') as file:
        yaml.dump(data_yaml, file)

# Assuming your data.yaml path is /content/Basketball-and-Hoop-Detection-1/data.yaml
#update_yaml_file('/content/Basketball-and-Hoop-Detection-1/data.yaml')
update_yaml_file('Warriors-vs-Cavs-2016-10\data.yaml')



In [ ]:

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)

data_path = r"C:\Users\ghadf\vscode_projects\venv_projects\Pytorch\YOLO\Warriors-vs-Cavs-2016-10\data.yaml"
# Use the model
model.train(data=data_path, epochs=20)  # train the model
metrics = model.val()  # evaluate model performance on the validation set


In [ ]:
results = model("https://www.juneauempire.com/wp-content/uploads/2018/08/15282699.jpg")  # predict on an image
path = model.export(format="onnx")  # export the model to ONNX format

In [ ]:
# Track with the model
#results = model.track(source="https://youtu.be/5zPxyWLls98?si=Bgew802YkfwJOC9T", show=True)
#results = model.track(source="https://youtu.be/5zPxyWLls98?si=Bgew802YkfwJOC9T", show=True, tracker="bytetrack.yaml")




In [ ]:
#load in data to be inferenced on
import yt_dlp

# List of YouTube video URLs you want to download
video_urls = [
    'https://youtu.be/5zPxyWLls98?si=Bgew802YkfwJOC9T'
    #'https://www.youtube.com/watch?v=BhVz5eoxpaw'
    #,'https://youtu.be/X-N9MEWYN8w'
]

# Replace with your desired output directory
output_path = '/content/Youtube_video/'

# Options for yt_dlp (YouTube Downloader)
ydl_opts = {
    'format': 'best',  # Select the best available format
    'outtmpl': output_path + '%(title)s.%(ext)s',  # Output file template
    'quiet': True,  # Suppress output messages
}

# Lists to store successful and failed download URLs
failed_downloads = []
successful_downloads = []

# Initialize YouTube Downloader with the provided options
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    for url in video_urls:
        try:
            ydl.download([url])  # Download the video
            print("Downloaded:", url)
            successful_downloads.append(url)
        except Exception as e:
            print(f"Failed to download {url}: {e}")
            failed_downloads.append(url)

# Print results
print("\n" * 4)
print("Failed downloads:", failed_downloads)
print("Successful downloads:", successful_downloads)


In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Open the video file
video_path = "content\Youtube_video\Stephen Curry hits three straight three-pointers vs. the Grizzlies (2016.04.13).mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
#model = YOLO('yolov8n.pt')

# Open the video file
video_path = video_path
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [ ]:
import threading
import cv2
from ultralytics import YOLO


def run_tracker_in_thread(filename, model, file_index):
    """
    Runs a video file or webcam stream concurrently with the YOLOv8 model using threading.

    This function captures video frames from a given file or camera source and utilizes the YOLOv8 model for object
    tracking. The function runs in its own thread for concurrent processing.

    Args:
        filename (str): The path to the video file or the identifier for the webcam/external camera source.
        model (obj): The YOLOv8 model object.
        file_index (int): An index to uniquely identify the file being processed, used for display purposes.

    Note:
        Press 'q' to quit the video display window.
    """
    video = cv2.VideoCapture(filename)  # Read the video file

    while True:
        ret, frame = video.read()  # Read the video frames

        # Exit the loop if no more frames in either video
        if not ret:
            break

        # Track objects in frames if available
        results = model.track(frame, persist=True)
        res_plotted = results[0].plot()
        cv2.imshow(f"Tracking_Stream_{file_index}", res_plotted)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    # Release video sources
    video.release()


# Load the models
model1 = YOLO('yolov8n-seg.pt')
model2 = model#YOLO('yolov8n-seg.pt')

# Define the video files for the trackers
video_file1 = video_path  # Path to video file, 0 for webcam
video_file2 = 0  # Path to video file, 0 for webcam, 1 for external camera

# Create the tracker threads
tracker_thread1 = threading.Thread(target=run_tracker_in_thread, args=(video_file1, model1, 1), daemon=True)
tracker_thread2 = threading.Thread(target=run_tracker_in_thread, args=(video_file2, model2, 2), daemon=True)

# Start the tracker threads
tracker_thread1.start()
tracker_thread2.start()

# Wait for the tracker threads to finish
tracker_thread1.join()
tracker_thread2.join()

# Clean up and close windows
cv2.destroyAllWindows()